In [1]:
import numpy as np
from numpy.linalg import norm as Enorm # Норма евклида 
from scipy.linalg import solve

# Положения источников 
A1 = np.array((0, -500, 0))
B1 = np.array((100, -500, 0))
A2 = np.array((0, 0, 0))
B2 = np.array((100, 0, 0))
A3 = np.array((0, 500, 0))
B3 = np.array((100, 500, 0))

# Положения приемников 
M1 = np.array((200, 0, 0))
N1 = np.array((300, 0, 0))
M2 = np.array((500, 0, 0))
N2 = np.array((600, 0, 0))
M3 = np.array((1000, 0, 0))
N3 = np.array((1100, 0, 0))

A = [A1, A2, A3]
B = [B1, B2, B3]
M = [M1, M2, M3]
N = [N1, N2, N3]

I = np.array((1,2, 3)) # Истенное значение силы тока каждого источника 
delta_I = np.zeros((3)) # Дельта смещения для поиска силы тока 
In = np.array((2.5, 3.5, 4.4))  # Начальное приближение для силы тока 
I_approx = In.copy()   # Примерное значение для регуляризации 

sigma = 0.1 # Проводимость среды 
alpha = 1e-9 # Параметр регуляризации 

# Потенциал на измерителе с учетом того, что источников тока 3 штуки 
# A,B - массив координат источника 
# M, N - точка измерителя 
# I - массив токов источников 
# sigma - коэффициент проводимости 
def V_AB_MN(A, B, M, N, I, sigma):

    res = 0.0

    for i in range(0, 3):
        const_val = I[i]/(2*np.pi * sigma)
        r_BM = Enorm(B[i]-M)
        r_AM = Enorm(A[i]-M)
        r_BN = Enorm(B[i]-N)
        r_AN = Enorm(A[i]-N)

        val2 = 1.0/r_BM - 1.0/r_AM
        val3 = 1.0/r_BN - 1.0/r_AN

        res = res + const_val*(val2 - val3) 
    
    return res # Значение напряжения на линии 

# Потенциал на измерителе с учетом того, что источников тока 3 штуки 
# A,B - массив координат источника 
# M, N - точка измерителя 
# I - массив токов источников начального приближения  
# sigma - коэффициент проводимости 
def dV_AB_MN_dI1(A, B, M, N, I, sigma):
    res = 0.0
    
    const_val = 1.0/(2*np.pi * sigma) 
    r_BM = Enorm(B[0]-M)
    r_AM = Enorm(A[0]-M)
    r_BN = Enorm(B[0]-N)
    r_AN = Enorm(A[0]-N)

    val2 = 1.0/r_BM - 1.0/r_AM
    val3 = 1.0/r_BN - 1.0/r_AN

    res = res + const_val*(val2 - val3) 

    return res # Значение напряжения на линии 


def dV_AB_MN_dI2(A, B, M, N, I, sigma):
    res = 0.0
    
    const_val = 1.0/(2*np.pi * sigma) 
    r_BM = Enorm(B[1]-M)
    r_AM = Enorm(A[1]-M)
    r_BN = Enorm(B[1]-N)
    r_AN = Enorm(A[1]-N)

    val2 = 1.0/r_BM - 1.0/r_AM
    val3 = 1.0/r_BN - 1.0/r_AN

    res = res + const_val*(val2 - val3) 

    return res # Значение напряжения на линии 



def dV_AB_MN_dI3(A, B, M, N, I, sigma):
    res = 0.0
    
    const_val = 1.0/(2*np.pi * sigma) 
    r_BM = Enorm(B[2]-M)
    r_AM = Enorm(A[2]-M)
    r_BN = Enorm(B[2]-N)
    r_AN = Enorm(A[2]-N)

    val2 = 1.0/r_BM - 1.0/r_AM
    val3 = 1.0/r_BN - 1.0/r_AN

    res = res + const_val*(val2 - val3) 

    return res # Значение напряжения на линии 


# Функционал невязки МНК
def F(I_, w, V):
    val = 0.0

    for i in range(0, 3):
        val = val + (w[i]*(V_AB_MN(A, B, M[i], N[i], I_, sigma) - V[i]))**2
    
    return val


# Например для 1 - ого источника имеем 
V = [V_AB_MN(A, B, Mi, Ni, I, sigma) for Mi, Ni in zip(M, N)]

w = [1/Vi for Vi in V] # Весовые коэффициенты 
dV = [dV_AB_MN_dI1, dV_AB_MN_dI2, dV_AB_MN_dI3]



Matr = np.zeros((3,3))
b = np.zeros((3))


for iteration in range(0, 100):
    descripency = F(In, w, V)
    print("Iter = ", iteration, " Невязка = " , descripency) 
    if descripency <= 1e-14 or iteration == 100-2:
        #print("Iter = ", iteration, " Невязка = " , descripency) 
        break

    # Генерация матрицы  
    for i in range(0, 3):
        for j in range(0, 3):
            sum = 0.0
            for k in range(0, 3):
                sum += w[k]**2* dV[i](A, B,M[k] ,N[k], In, sigma)*dV[j](A, B,M[k] ,N[k], In, sigma)
            Matr[i][j]= sum
        
        Matr[i][i] = Matr[i][i] + alpha # Регуляризация системы уравнений 

    for i in range(0, 3):
        val = 0.0
        for j in range(0, 3):
            val = val + (w[j]**2)*dV[i](A, B, M[j], N[j],In, sigma)*(V_AB_MN(A, B, M[j], N[j], In, sigma) - V[j])
        b[i] = -val
        # Регуляризация 
        b[i] = b[i] - alpha*(In[i] - I_approx[i])
    
    #print(b)
    delta_I = solve(Matr, b.reshape((3,1)))
    delta_I = delta_I.reshape(3)
    In = In + delta_I
   


print("alpha = ", alpha)
print("Истенный ток: ", I)
print("Расчитанный: ", In)




Iter =  0  Невязка =  1.6645588660625714
Iter =  1  Невязка =  1.3785636603586952e-16
alpha =  1e-09
Истенный ток:  [1 2 3]
Расчитанный:  [1.05000005 1.99999999 2.95000005]
